# Segmenting Images using YOLO model

## Training Semantic Segmentation Model

## Using the model to get segmented images

Loading libraries

In [2]:
from ultralytics import YOLO
import torch
import cv2
import numpy as np
import os

model = YOLO("../models/segmentation_model.pt")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Segmenting the images

In [8]:
image_folder = "../images"
output_folder = "../segmented_images"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(image_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)

        results = model.predict(image_path, device=device, max_det=1, conf=0.85, save=False, verbose=False)
        for result in results:
            for c in result:
                polygons = c.masks.xy[0].astype(int).tolist()
                poly_np = np.array(polygons, dtype=np.int32)
                epsilon = 0.0005 * cv2.arcLength(poly_np, True)
                approx = cv2.approxPolyDP(poly_np, epsilon, True)
                polygons = approx.reshape(-1, 2).tolist()

                # Create mask
                mask = np.zeros(image.shape[:2], dtype=np.uint8)
                cv2.fillPoly(mask, [np.array(polygons, dtype=np.int32)], 255)

                # Mask image
                masked_img = cv2.bitwise_and(image, image, mask=mask)

                # Crop to bounding rect
                x, y, w, h = cv2.boundingRect(np.array(polygons, dtype=np.int32))
                cropped = masked_img[y:y+h, x:x+w]

                # Save
                save_path = os.path.join(output_folder, filename)
                cv2.imwrite(save_path, cropped)
                
        torch.cuda.empty_cache()